##  import Beautifulsoup, request, selenium 사용

In [13]:
import requests
from bs4 import BeautifulSoup, element
import chardet
from datetime import datetime, timedelta
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
import urllib3

## 네이버 크롤링 위해 고정 header 사용

In [27]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
resultList = []

## Chrome 드라이버 생성 , headless 모드(웹브라우저 실행없이)로 실행, SSSL 인증서 오류 무시

In [15]:
chrome_options = Options()
chrome_options.add_argument("--headless")  
chrome_options.add_argument('--ignore-certificate-errors') 

#   크롤링 함수 정의 
### 뉴스 검색 사이트 URL 부분의 검색명에 {회사 이름}를 넣어줌  
### 뉴스 제목, 뉴스 날짜, 언론사, 뉴스 내용을 크롤링

In [38]:
def start_crawl(start_date, end_date,company):
    global news_date
    start_date_nospace = start_date.replace('.', '')
    end_date_nopspace = end_date.replace('.', '')
    current_page = 1
    searching = True

    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(5)

    search_urlback = f"{start_date}&de={end_date}&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{start_date_nospace}to{end_date_nopspace},a:all&start=1"
    search_urlfront = f"https://search.naver.com/search.naver?where=news&query={company}&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds="
    search_url = search_urlfront + search_urlback
    print(search_url)
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 새로운 컨텐츠가 로드될 때까지 대기
    while True:
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            last_height = new_height
            continue 
        else:
            last_height = new_height
            break

    naver_html_src = driver.page_source
    naver_soup = BeautifulSoup(naver_html_src, 'html.parser')
    pages = naver_soup.find('div', {'class' : 'sc_page_inner'})
    news_list = naver_soup.select('li[class="bx"]')

    for news in news_list:
        time.sleep(2)
        
        # 뉴스 제목
        try :
            news_title = news.find('a', attrs={'class':'news_tit'}).get('title')
        except AttributeError:
            news_title = "No title available"
        news_url = news.find('a', attrs={'class':'news_tit'}).get('href')

        # 뉴스 날짜 
        find_date = news.find_all('span', attrs={'class':'info'})
        if len(find_date) > 1:
            # if find_date == "4주 전":
            #     news_date = 20240212
            # elif find_date == "3주 전":
            #     news_date = 20240219
            # elif find_date == "2주 전":
            #     news_date = 20240226
            # elif find_date == "1주 전":
            #     news_date = 20240303
            # elif find_date == "*일 전":
            #     news_date = 20240312
            # else :                
            news_date = find_date[1].text
        else: 
            news_date = find_date[0].text
            
        # 뉴스 발행사 
        try:
            news_publisher = news.find('a', attrs={'class':'info press'}).text
        except:
            try:
                news_publisher = news.find('a', attrs={'class':'info'}).text
            except:
                continue
            
        # 뉴스 내용
        news_summary = news.find('a', attrs={'class':'api_txt_lines dsc_txt_wrap'}).text
        news_summary = news_summary[0:50]
        
        news_text = []
        try:
            news_response = requests.get(news_url, headers=headers)
        except :
            continue

        if news_summary in str(news_response.text):
            news_soup = BeautifulSoup(news_response.text, 'html.parser')
        else:
            news_soup = BeautifulSoup(news_response.content.decode('euc-kr', 'replace'), 'html.parser')
            
        news_content = str(news_soup.text).split('\n')
        news_index = [i for i, s in enumerate(news_content) if news_summary in s]
        
        if news_index:
            news_index = news_index[0]
            punctuations = ['.', ',', '?', '!']
            count = 0
            if news_index != 0:
                for i in reversed(range(0, news_index)):
                    if any(text in news_content[i] for text in punctuations): 
                        news_text.append(news_content[i])
                        count = 0
                        #print("i", i, "count", count)
                        #print(news_content[i])
                    else:
                        count +=1
                    if count > 3:
                        break

            # print(news_index)

            if news_text:
                news_text.reverse()
            count = 0
            news_text.append(news_content[news_index])
            for i in range(news_index, len(news_content)):
                
                if any(text in news_content[i] for text in punctuations): 
                    news_text.append(news_content[i])
                    count = 0

                else:
                    count +=1
                if count > 3:
                    break
            
            news_text = ' '.join(news_text)
        resultList.append([news_date, news_title, news_text, news_publisher])
        df = pd.DataFrame(resultList)

        # updated_df = pd.concat([existing_df, df], ignore_index=True)
        df.to_csv(f'news_{company}1.csv', encoding='utf-8-sig', index=False)

    driver.quit()

    return news_text, news_date

# 함수 실행
### 시작날짜, 끝날짜, 회사이름
### 중간에 언론사 or 네이버 서버에 의해 네트워크 중지되었다는 오류 발생할 수도 있음-=> .py로 실행
### 

In [39]:
# 실행
start_crawl('20240101','20240102','동진쎄미켐')

https://search.naver.com/search.naver?where=news&query=동진쎄미켐&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=20240101&de=20240102&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20240101to20240102,a:all&start=1


Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq\\backend\\cython\\checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 


KeyboardInterrupt: 